In [25]:
import numpy as np #Imports NumPy, a library used for handling numbers, arrays, and mathematical operations efficiently.
import pandas as pd # Imports Pandas, a powerful tool for working with data in table format (like Excel sheets), great for data analysis.
import nltk # Imports NLTK (Natural Language Toolkit) — a library for text processing and natural language tasks like tokenization, stopwords, etc.
from nltk.sentiment.vader import SentimentIntensityAnalyzer # Imports VADER sentiment analyzer from NLTK. It's used to detect whether text has positive, negative, or neutral sentiment.
import re
from textblob import TextBlob  # Fixed typo: 'tectblob' → 'textblob'
from wordcloud import WordCloud
  # Fixed syntax and capitalization

import seaborn as sns
import matplotlib.pyplot as plt

import cufflinks as cfN
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)  # Fixed typo: 'inint' → 'init'
cf.go_offline()

import plotly.graph_objs as go
from plotly.subplots import make_subplots  # Fixed typo: 'subplot' → 'subplots'

import warnings 
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

pd.set_option('display.max_columns', None)

In [14]:
df.head() #head used to see top 5 rowa of my data set

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4,No issues.,23-07-2014,138,0,0,0,0,0.0,0.0
1,1,0mie,5,"Purchased this for my device, it worked as adv...",25-10-2013,409,0,0,0,0,0.0,0.0
2,2,1K3,4,it works as expected. I should have sprung for...,23-12-2012,715,0,0,0,0,0.0,0.0
3,3,1m2,5,This think has worked out great.Had a diff. br...,21-11-2013,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5,"Bought it with Retail Packaging, arrived legit...",13-07-2013,513,0,0,0,0,0.0,0.0


In [15]:
df = pd.read_csv("amazon.csv")

In [16]:
df = df.sort_values("wilson_lower_bound",ascending = False)
df.drop('Unnamed: 0', inplace = True, axis=1)
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,I have tested dozens of SDHC and micro-SDHC ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,NOTE: please read the last update (scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,"If your card gets hot enough to be painful, it...",09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,Sandisk announcement of the first 128GB micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [17]:
def missing_values_analysis(df):
    na_columns_ = [col for col in df.columns if df[col].isnull().sum() > 0]
    n_miss = df[na_columns_].isnull().sum().sort_values(ascending=True)
    ratio_ = (df[na_columns_].isnull().sum() / df.shape[0] * 100).sort_values(ascending=True)
    missing_df = pd.concat([n_miss, np.round(ratio_, 2)], axis=1, keys=['Missing Values', 'Ratio (%)'])
    return missing_df

def check_dataframe(df, head=5, tail=5):
    print("SHAPE".center(82, '"'))
    print('ROWS: {}'.format(df.shape[0]))
    print('COLUMNS: {}'.format(df.shape[1]))
    
    print("TYPES".center(82, '"'))
    print(df.dtypes)
    
    print("HEAD".center(82, '"'))
    print(df.head(head))
    
    print("TAIL".center(82, '"'))
    print(df.tail(tail))
    
    print("MISSING VALUES".center(82, '"'))
    print(missing_values_analysis(df))
    
    print("DUPLICATED VALUES".center(82, '"'))
    print(df.duplicated().sum())
    
    print("QUANTILES (Numerical Columns Only)".center(82, '"'))
    numeric_df = df.select_dtypes(include=[np.number])
    print(numeric_df.quantile([0, 0.05, 0.95, 0.99, 1]).T)

# Example usage:
# check_dataframe(df)

check_dataframe(df)



""""""""""""""""""""""""""""""""""""""SHAPE"""""""""""""""""""""""""""""""""""""""
ROWS: 4915
COLUMNS: 11
""""""""""""""""""""""""""""""""""""""TYPES"""""""""""""""""""""""""""""""""""""""
reviewerName             object
overall                   int64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object
"""""""""""""""""""""""""""""""""""""""HEAD"""""""""""""""""""""""""""""""""""""""
                 reviewerName  overall  \
2031     Hyoun Kim "Faluzure"        5   
3449        NLee the Engineer        5   
4212              SkincareCEO        1   
317   Amazon Customer "Kelly"        1   
4672                  Twister        5   

                                             reviewText  reviewTime  day_diff  \
2031  [[ UPDATE - 6/19/2014 

In [18]:
def check_class(dataframe):
    nunique_df = pd.DataFrame({'Variable': dataframe.columns,
                               'Classes': [dataframe[i].nunique() \
                               for i in dataframe.columns]})
    nunique_df = nunique_df.sort_values('Classes', ascending = False)
    nunique_df = nunique_df.reset_index(drop = True)
    return nunique_df

check_class(df)

,Variable,Classes
0,reviewText,4912
1,reviewerName,4594
2,reviewTime,690
3,day_diff,690
4,wilson_lower_bound,40
5,score_average_rating,28
6,score_pos_neg_diff,27
7,total_vote,26
8,helpful_yes,23
9,helpful_no,17


In [19]:
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

constraints = ['#B34D22', '#EBE00C', '#1FEB0C', '#0C92EB', '#EB0CD5']

def categorical_variable_summary(df):
    column_name = 'overall'  # Column name set here
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Countplot', 'Percentage'),
        specs=[[{"type": "xy"}, {'type': 'domain'}]]
    )

    counts = df[column_name].value_counts()
    labels = [str(i) for i in counts.index]
    values = counts.values.tolist()

    # Bar Chart
    fig.add_trace(
        go.Bar(
            y=values,
            x=labels,
            text=values,
            textfont=dict(size=14),
            name=column_name,
            textposition='auto',
            showlegend=False,
            marker=dict(
                color=constraints[:len(values)],
                line=dict(color='#DBE6EC', width=1)
            )
        ),
        row=1, col=1
    )

    # Pie Chart
    fig.add_trace(
        go.Pie(
            labels=labels,
            values=values,
            textfont=dict(size=18),
            textposition='auto',
            showlegend=False,
            name=column_name,
            marker=dict(colors=constraints[:len(values)])
        ),
        row=1, col=2
    )

    fig.update_layout(
        title=dict(
            text='Distribution of Overall',
            y=0.9,
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
        template='plotly_white'
    )

    iplot(fig)

# Call the function with your DataFrame
categorical_variable_summary(df)



In [20]:
categorical_variable_summary(df)



In [21]:
df.reviewText.head()

2031    [[ UPDATE - 6/19/2014 ]]So my lovely wife boug...
3449    I have tested dozens of SDHC and micro-SDHC ca...
4212    NOTE:  please read the last update (scroll to ...
317     If your card gets hot enough to be painful, it...
4672    Sandisk announcement of the first 128GB micro ...
Name: reviewText, dtype: object

In [22]:
review_example = df.reviewText[231]
review_example

"I needed this for my Samsung Galley tablet. It's doing the job it was needed for. I really didn't need the adapter, but most all micro SD cards come with them."

In [23]:
review_example = re.sub("[^a-zA-Z]",'',review_example)
review_example

'IneededthisformySamsungGalleytabletItsdoingthejobitwasneededforIreallydidntneedtheadapterbutmostallmicroSDcardscomewiththem'

In [24]:
review_example = review_example.lower().split()
review_example

['ineededthisformysamsunggalleytabletitsdoingthejobitwasneededforireallydidntneedtheadapterbutmostallmicrosdcardscomewiththem']

In [13]:
rt = lambda x: re.sub("[^a-zA-Z]", ' ', str(x))
df["reviewText"] = df["reviewText"].map(rt)
df["reviewText"] = df["reviewText"].str.lower()
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,i have tested dozens of sdhc and micro sdhc ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,note please read the last update scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,if your card gets hot enough to be painful it...,09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,sandisk announcement of the first gb micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import pandas as pd

# Extract polarity and subjectivity from TextBlob
df[['polarity', 'subjectivity']] = df['reviewText'].apply(lambda text: pd.Series(TextBlob(text).sentiment))

# Initialize VADER analyzer
analyzer = SentimentIntensityAnalyzer()

# Classify sentiment
def classify_sentiment(text):
    score = analyzer.polarity_scores(text)
    if score['neg'] > score['pos']:
        return "Negative"
    elif score['pos'] > score['neg']:
        return "Positive"
    else:
        return "Neutral"

# Apply classification
df['sentiment'] = df['reviewText'].apply(classify_sentiment)


In [17]:
df[df['sentiment']=='Positive'].sort_values("wilson_lower_bound",
                                            ascending = False).head(5)

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,sentiment
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544,0.163859,0.562259,Positive
3449,NLee the Engineer,5,i have tested dozens of sdhc and micro sdhc ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519,0.103870,0.516435,Positive
4212,SkincareCEO,1,note please read the last update scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139,0.212251,0.505394,Positive
317,"Amazon Customer ""Kelly""",1,if your card gets hot enough to be painful it...,09-02-2012,1033,422,73,495,349,0.852525,0.818577,0.143519,0.494207,Positive
4672,Twister,5,sandisk announcement of the first gb micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109,0.172332,0.511282,Positive
